### BERT Pre-train on GPU

Succesfully run on Google Colab GPU as of 6/6/2020 at 9:45am    \
Saved as model_ft6.pth      \
2 Epochs, Batch Size 32       

In [ ]:
import BERT_Project_Package as bpp

#For all your pickle needs
import pickle

#General stuff
import pandas as pd
import numpy as np

#For Train and Test Split
import random

#For Running BERT
from transformers import BertTokenizer, BertForPreTraining, BertModel, TFBertForMaskedLM, BertConfig, BertForMaskedLM
import torch

#Data preparation for Fine tuning
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer

#For fine-tuning BERT
import torch
from transformers import BertModel

#Initialize optimizer for pre-training
from transformers import AdamW

#For splitting tuples
import ast

#For progress bar
from tqdm import tqdm

In [ ]:
#initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
#initialize model
model = BertForMaskedLM.from_pretrained('bert-base-cased')

In [ ]:
#Load Text Data
import json
with open('easier.json') as d:
    x_dict = json.load(d)

In [ ]:
train, validate, test = bpp.trainvaltest(x_dict)

In [ ]:
#Create necessary data input for both NSP and MLM task
my_inputs = bpp.nsp_mlm(train)
my_inputs_test = bpp.nsp_mlm(validate)

In [ ]:
dataset = bpp.TorchDataset(my_inputs)

In [ ]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size = 16, shuffle = True)

In [ ]:
#Set the training parameters
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
#Set the device, we'll want to use GPU for our training, will have to purchase
#This is what that looks like: https://cloud.google.com/tpu/docs/quickstart
model.to(device) #MLM

In [ ]:
#Activate our model's training mode
model.train() #MLM

In [ ]:
#Set learn rate
optim = AdamW(model.parameters(), lr = 1e-5)

In [ ]:
#set this as needed, we'll want to do more
epochs = 2

#set training loop
for epoch in range(epochs):
    loop = tqdm(dataloader, leave = True)
    for batch in loop: 
    #for every loop calculate the gradient
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device) #this will put this to GPU if device = GPU
        token_type_ids = batch['token_type_ids'].to(device) #for NSP and MLM, or NSP Only
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        #Now we have out ids, our token ids (for NSP and MLM, or NSP only), our masked data, and our true labels, we can process
        #outputs = model(input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask, labels = labels) for MLM only
        outputs = model(input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask, labels = labels)
        loss = outputs.loss
        #loss is difference between our prediction and our labels
        #Calculate loss for every parameter in our model to calculate gradient using optimizer
        loss = outputs.loss
        loss.backward()
        optim.step()
        #takes a step to optimize based on our calculated loss

        #For each loop, show the epoch and the loss
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss = loss.item())

In [ ]:
torch.save(model, 'model_ft6.pth')